1) Carrega bibliotecas e DataFrame; configura visualização

In [1]:
# Manipulação e Análise
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Geocodificação e APIs
import requests
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Modelagem Preditiva
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split

# Salvar dataframes
import os


# Configura visualização
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
sns.set_theme()

caminho_dados = '../dados/primarios/acidentes.csv'

try:
    df = pd.read_csv(caminho_dados, sep=';', encoding='utf-8')
    print("CSV original lido com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo '{caminho_dados}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

CSV original lido com sucesso!


2) Entende DataFrame

In [2]:
# Lista as cinco primeira linhas
print("\nCinco primeiras linhas:")
print(df.head())

# Mostra o número de linhas e colunas
print("\nDimensões do Dataset (linhas, colunas)")
print(df.shape)

# Mostra tipo de dados, valores não nulos
print("\nInformações gerais do DataFrame:")
df.info()

# Conta valores ausentes por coluna
print("\nContagem de Valores Ausentes por Coluna")
print(df.isnull().sum())



Cinco primeiras linhas:
         data_extracao  predial1  queda_arr                 data  feridos  \
0  2025-06-01 01:33:13       0.0        0.0  2020-10-17 00:00:00        1   
1  2025-06-01 01:33:13     598.0        0.0  2020-01-01 00:00:00        1   
2  2025-06-01 01:33:13    1271.0        0.0  2020-01-01 00:00:00        1   
3  2025-06-01 01:33:13    1901.0        0.0  2020-01-02 00:00:00        2   
4  2025-06-01 01:33:13    3302.0        0.0  2020-01-02 00:00:00        1   

   feridos_gr  mortes  morte_post  fatais  auto  taxi  lotacao  onibus_urb  \
0           0       0           0       0     3     0        0           0   
1           0       0           0       0     0     1        0           0   
2           1       0           0       0     1     0        0           0   
3           0       0           0       0     0     0        0           0   
4           0       0           0       0     1     0        0           0   

   onibus_met  onibus_int  caminhao  moto  

3) Limpa nomes colunas; remove 'data_extracao', 'consorcio', 'latitude' e 'longitude'

In [3]:
# Remove espaços em branco no início/fim dos nomes das colunas (boa prática)
df.columns = df.columns.str.strip()

# Remove colunas não utilizadas
df = df.drop(columns=['data_extracao', 'consorcio', 'latitude', 'longitude'], errors='ignore')

# Lista nome de colunas
print(df.columns)


Index(['predial1', 'queda_arr', 'data', 'feridos', 'feridos_gr', 'mortes',
       'morte_post', 'fatais', 'auto', 'taxi', 'lotacao', 'onibus_urb',
       'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta',
       'outro', 'cont_vit', 'ups', 'patinete', 'idacidente', 'log1', 'log2',
       'tipo_acid', 'dia_sem', 'hora', 'noite_dia', 'regiao'],
      dtype='object')


4) Transforma tipos; remove inválidos

In [4]:
# Converte 'data' para datetime
df['data'] = pd.to_datetime(df['data'], errors='coerce')

# Remove inválidos 'data', 'hora', 'log1', 'regiao'
df = df.dropna(subset=['data', 'hora', 'log1', 'regiao'])

# Remove data fora do escopo (2020-2025)
df = df[(df['data'] >= '2020-01-01') & (df['data'] <= '2025-04-01')]
print("Datas fora do escopo (2020-2025) removidas com sucesso.")

# Remove chaves duplicadas
df = df.drop_duplicates(subset='idacidente')

# Transforma colunas tipos de colunas em Categoria
col_cat = ['regiao', 'tipo_acid', 'dia_sem', 'noite_dia']
df[col_cat] = df[col_cat].astype('category')

# Transforma colunas tipos de colunas Inteiro
col_int = [c for c in [
  'queda_arr', 'feridos', 'feridos_gr', 'mortes',
  'morte_post', 'fatais', 'auto', 'taxi', 'lotacao', 'onibus_urb',
  'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta',
  'outro', 'cont_vit', 'ups', 'patinete', 'idacidente', 'predial1'
  ] if c in df.columns]
for c in col_int:
    df[c] = pd.to_numeric(df[c], errors='coerce').astype('Int32')

# Transforma colunas tipos de colunas em Float
col_float = [c for c in ['longitude', 'latitude'] if c in df.columns]
for c in col_float:
    df[c] = pd.to_numeric(df[c], errors='coerce').astype('float32')

# Transforma colunas tipos de colunas em String
col_str = [c for c in ['log1', 'log2'] if c in df.columns]
for c in col_str:
    df[c] = df[c].astype('string')

# Transforma coluna 'hora' para timedelta
df['hora'] = pd.to_timedelta(df['hora'], errors='coerce')

     
print("\nInformações após:")
df.info()
df.describe(include='all')
df.head()

Datas fora do escopo (2020-2025) removidas com sucesso.

Informações após:
<class 'pandas.core.frame.DataFrame'>
Index: 68837 entries, 0 to 69520
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   predial1    64799 non-null  Int32          
 1   queda_arr   68837 non-null  Int32          
 2   data        68837 non-null  datetime64[ns] 
 3   feridos     68837 non-null  Int32          
 4   feridos_gr  68837 non-null  Int32          
 5   mortes      68837 non-null  Int32          
 6   morte_post  68837 non-null  Int32          
 7   fatais      68837 non-null  Int32          
 8   auto        68837 non-null  Int32          
 9   taxi        68837 non-null  Int32          
 10  lotacao     68837 non-null  Int32          
 11  onibus_urb  68837 non-null  Int32          
 12  onibus_met  68837 non-null  Int32          
 13  onibus_int  68837 non-null  Int32          
 14  caminhao    68837 non-null  Int3

,predial1,queda_arr,data,feridos,feridos_gr,mortes,morte_post,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao
0,0,0,2020-10-17,1,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,1,5,0,190816,R MARCOS MOREIRA,R GASTON ENGLERT,ABALROAMENTO,SÁBADO,0 days 19:00:00,NOITE,NORTE
1,598,0,2020-01-01,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,5,0,669089,AV BENTO GONCALVES,<NA>,ABALROAMENTO,QUARTA-FEIRA,0 days 03:00:00,NOITE,LESTE
2,1271,0,2020-01-01,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,0,669097,AV INDEPENDENCIA,<NA>,ATROPELAMENTO,QUARTA-FEIRA,0 days 23:00:00,NOITE,LESTE
3,1901,0,2020-01-02,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,5,0,669098,AV EDUARDO PRADO,<NA>,ATROPELAMENTO,QUINTA-FEIRA,0 days 00:05:00,NOITE,SUL
4,3302,0,2020-01-02,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,5,0,669099,AV TERESOPOLIS,<NA>,ABALROAMENTO,QUINTA-FEIRA,0 days 09:00:00,DIA,SUL


5) Cria novas variáveis e DataFrames necessários

In [11]:
# Cria coluna 'hora_int'
df['hora_int'] = df['hora'].dt.components['hours']

# Cria coluna 'data_hora'
df['data_hora'] = df['data']+ df['hora']

# Salva dataframe tratado
df.to_parquet("../dados/intermediarios/acidentes_tratado.parquet", index=False, engine="pyarrow")

# print(df.head())
# print(df.tail())


6) Puxa dados de precipitação baseado nas regiões

In [ ]:
# Coordenadas centrais das regiões
coord = {
    "NORTE":  (-29.987, -51.165),
    "LESTE":  (-30.040, -51.160),
    "CENTRO": (-30.027, -51.220),
    "SUL":    (-30.120, -51.230)
}

# Define colunas necessárias
chuva = ["idacidente", "data", "hora_int", "data_hora", "regiao"]
anos = [2020, 2021, 2022, 2023, 2024]
dfs_all = []

# garante a pasta para salvar
os.makedirs("../dados/intermediarios", exist_ok=True)

# Cria 4 dataframes (um por região)
df_norte  = df[df["regiao"] == "NORTE"][chuva].copy()
df_leste  = df[df["regiao"] == "LESTE"][chuva].copy()
df_centro = df[df["regiao"] == "CENTRO"][chuva].copy()
df_sul    = df[df["regiao"] == "SUL"][chuva].copy()

# Cria dataframes por ano
dfs_norte = {ano: df_norte[df_norte["data"].dt.year == ano] for ano in range(2020, 2025)}
dfs_leste = {ano: df_leste[df_leste["data"].dt.year == ano] for ano in range(2020, 2025)}
dfs_centro = {ano: df_centro[df_centro["data"].dt.year == ano] for ano in range(2020, 2025)}
dfs_sul = {ano: df_sul[df_sul["data"].dt.year == ano] for ano in range(2020, 2025)}


list(dfs_centro.values())
print('--------------------------------------------------\n')

"""
# Consulta API Open Meteo
def chuva_regiao_ano(regiao, lat, lon, ano):
    data_inicio = f"{ano}-01-01"
    data_fim = f"{ano}-12-31"
    
    url = (
        "https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={data_inicio}&end_date={data_fim}"
        "&hourly=precipitation"
        "&timezone=America/Sao_Paulo"
    )
    
    resposta = requests.get(url).json()
    
    if "hourly" in resposta and "time" in resposta["hourly"]:
        df_meteo = pd.DataFrame({
            "time": pd.to_datetime(resposta["hourly"]["time"], errors="coerce"),
            "precipitation": resposta["hourly"]["precipitation"],
        })
        df_meteo["regiao"] = regiao
        return df_meteo
    else:
        print(f"⚠️ Sem dados para {regiao}-{ano}")
        return pd.DataFrame(columns=["time","precipitation","regiao"])


for ano in anos:
    for regiao, (lat, lon) in coord.items():
        print(f"📥 Baixando clima {regiao}-{ano}...")
        df_meteo = chuva_regiao_ano(regiao, lat, lon, ano)
        
        if not df_meteo.empty:
            # salva parquet individual
            fname = f"../dados/intermediarios/clima_{regiao}_{ano}.parquet"
            df_meteo.to_parquet(fname, index=False, engine="pyarrow")
            print(f"✅ salvo {fname} ({len(df_meteo)} linhas)")
            
            dfs_all.append(df_meteo)
"""

--------------------------------------------------



'\n# Consulta API Open Meteo\ndef chuva_regiao_ano(regiao, lat, lon, ano):\n    data_inicio = f"{ano}-01-01"\n    data_fim = f"{ano}-12-31"\n\n    url = (\n        "https://archive-api.open-meteo.com/v1/archive"\n        f"?latitude={lat}&longitude={lon}"\n        f"&start_date={data_inicio}&end_date={data_fim}"\n        "&hourly=precipitation"\n        "&timezone=America/Sao_Paulo"\n    )\n\n    resposta = requests.get(url).json()\n\n    if "hourly" in resposta and "time" in resposta["hourly"]:\n        df_meteo = pd.DataFrame({\n            "time": pd.to_datetime(resposta["hourly"]["time"], errors="coerce"),\n            "precipitation": resposta["hourly"]["precipitation"],\n        })\n        df_meteo["regiao"] = regiao\n        return df_meteo\n    else:\n        print(f"⚠️ Sem dados para {regiao}-{ano}")\n        return pd.DataFrame(columns=["time","precipitation","regiao"])\n\n\nfor ano in anos:\n    for regiao, (lat, lon) in coord.items():\n        print(f"📥 Baixando clima {reg